In [28]:
import requests
import pandas as pd
import plotly.express as px

In [132]:
# Functions to query USAspending.gov API for recent grants by county FIPS code
# TODO: make this a separate function (in utils?) and import it
# to get more queries, go to USA spending website, filter what you want, then inspect page, look at network tab, and copy the request payload and header

url_FIPS = "https://www2.census.gov/geo/docs/reference/codes/files/national_county.txt"
df_county_fips = pd.read_csv(url_FIPS, header=None, dtype=str)
df_county_fips.columns = [
    "State",
    "State FIPS",
    "County FIPS",
    "County Name",
    "Class Code",
]
df_county_fips


def fips_to_geoname(fips):
    fips = str(fips)
    county_fips = fips[-3:]
    state_fips = fips[:-3]
    state_code = df_county_fips[df_county_fips["State FIPS"] == state_fips][
        "State"
    ].values[0]
    # find county that matches state and county fips
    county_name = df_county_fips[
        (df_county_fips["County FIPS"] == county_fips)
        & (df_county_fips["State FIPS"] == state_fips)
    ]["County Name"].values[0]
    return county_name + ", " + state_code


def fips_to_county_state(fips):
    fips = str(fips)
    county_fips = fips[-3:]
    state_fips = fips[:-3]
    state_code = df_county_fips[df_county_fips["State FIPS"] == state_fips][
        "State"
    ].values[0]
    return state_code, county_fips


def df_from_api(base_url, payload):
    headers = {"Content-Type": "application/json"}

    # run api call
    response = requests.post(base_url, headers=headers, json=payload)
    response_json = response.json()

    # convert to dataframe, keeping only relevant columns
    df = pd.DataFrame(response_json["results"])
    # df = df[["Award Amount", "Awarding Agency", "Recipient Name"]]
    return df


def grant_df_from_fips(fips):
    state_code, county_fips = fips_to_county_state(fips)
    base_url = "https://api.usaspending.gov/api/v2/search/spending_by_award/"

    # options for api call
    payload = {
        "filters": {
            "time_period": [
                {"start_date": "2018-10-01", "end_date": "2019-09-30"},
                {"start_date": "2022-10-01", "end_date": "2023-09-30"},
            ],
            "award_type_codes": ["02", "03", "04", "05"],
            "place_of_performance_locations": [
                {"country": "USA", "state": state_code, "county": county_fips}
            ],
        },
        "fields": [
            "Award ID",
            "Recipient Name",
            "Start Date",
            "End Date",
            "Award Amount",
            "Total Outlays",
            "Description",
            "def_codes",
            "COVID-19 Obligations",
            "COVID-19 Outlays",
            "Infrastructure Obligations",
            "Infrastructure Outlays",
            "Awarding Agency",
            "Awarding Sub Agency",
            "Award Type",
            "recipient_id",
            "prime_award_recipient_id",
        ],
        "page": 1,
        "limit": 100,
        "sort": "Award Amount",
        "order": "desc",
        "subawards": False,
    }

    df = df_from_api(base_url, payload)

    return df

In [147]:
# Running the grants query for a specific FIPS code, and plotting the results
GeoFIPS = "25011"  # "36061"#"08013"#"39035"#"02188"#"09009"#"01003" #"01089"#"25017"
df = grant_df_from_fips(GeoFIPS)

# make a pie chart of the agency by amount
df["Award Amount"] = pd.to_numeric(df["Award Amount"])
title = "Recent grants to " + fips_to_geoname(GeoFIPS) + " by Agency"
fig = px.pie(df, values="Award Amount", names="Awarding Agency", title=title)
fig.update_traces(textinfo="label+value")
fig.show()
# df